### instantiating services

In [ ]:
from apps.homebase.abis import wrapperAbi, daoAbiGlobal,tokenAbiGlobal
from apps.homebase.paper import Paper
from datetime import datetime
import time
from firebase_admin import initialize_app
from firebase_admin import firestore,credentials
from web3 import Web3
import re
cred = credentials.Certificate('homebase.json')
initialize_app(cred)
db = firestore.client()
networks=db.collection("contracts")
ceva=networks.document("Etherlink-Testnet").get()
wrapper_address = ceva.to_dict()['wrapper']
print("wrapper address :" + str(wrapper_address))
rpc="https://node.ghostnet.etherlink.com"
web3 = Web3(Web3.HTTPProvider(rpc))
papers={}
daos=[]
if web3.is_connected():
    print("node connected")
else:
    print("node connection failed!")

daos_collection=db.collection('idaosEtherlink-Testnet')
docs = list(daos_collection.stream())
dao_addresses = [doc.id for doc in docs]

for doc in docs:
    obj=doc.to_dict()
    try:
        p=Paper(address=obj['token'],kind="token", daos_collection=daos_collection,db=db,  web3=web3, dao=doc.id)
        dao=Paper(address=obj['address'],kind="dao", daos_collection=daos_collection,db=db,  web3=web3, dao=doc.id)
    except Exception as e:
        print("one DAO contract can't parse correctly: "+str(e))
    papers.update({obj['token']:p})
    papers.update({obj['address']:dao})

event_signatures = {
    web3.keccak(text="NewDaoCreated(address,address,address[],uint256[],string,string,string,uint256,address,string[],string[])").hex(): "NewDaoCreated",
    "0x3134e8a2e6d97e929a7e54011ea5485d7d196dd5f0ba4d4ef95803e8e3fc257f":"DelegateChanged",
    "0x7d84a6263ae0d98d3329bd7b46bb4e8d6f98cd35a7adb45c274c8b7fd5ebd5e0":"ProposalCreated"
}

papers.update({wrapper_address:Paper(address=wrapper_address,kind="wrapper",daos_collection=daos_collection,db=db,  web3=web3, dao=doc.id )})
listening_to_addresses=[wrapper_address]
listening_to_addresses=listening_to_addresses+list(papers.keys())


counter=0
processed_transactions = set()
heartbeat=0
print(f"Listening for {len(event_signatures)} events on {len(papers.items())} contracts...")

while True:
    heartbeat+=1
    try:
        latest=web3.eth.block_number
        first=latest-4
        logs = web3.eth.get_logs({
            "fromBlock": first,
            "toBlock": latest,
            "address": listening_to_addresses,
            # "topics": [[*event_signatures.keys()], None]
        })
        for log in logs:
            tx_hash = log["transactionHash"].hex()
            if tx_hash in processed_transactions:
                print("already did this one")
                continue  # Skip duplicate
            contract_address = log["address"]
            event_signature = log["topics"][0].hex()
            print("is this even working?")
            event_name = event_signatures[f"0x{event_signature}"]
            print(f"Event: {event_name}, Contract: {contract_address}")
            new_contract_addresses=papers[contract_address].handle_event(log, web3=web3)
            print("we did pass this")
            if new_contract_addresses != None:
                dao_address=new_contract_addresses[0]
                token_address=new_contract_addresses[1]
                listening_to_addresses=listening_to_addresses+[dao_address]+[token_address]
                papers.update({token_address:Paper(address=token_address,kind="token", dao=dao_address )})
            processed_transactions.add(tx_hash)
    except Exception as e:
        print("something went wrong "+str(e))

    if heartbeat%50==0:
        print("heartbeat: "+str(heartbeat))

    time.sleep(3)
    

wrapper address :0xDF38eD7d9DcF15577F0E6eb619b945669F1f74b1
node connected
Listening for 3 events on 21 contracts...
is this even working?
Event: ProposalCreated, Contract: 0x09166B4cA9572648B42A2A644D0c0daF4888Df48
something went wrong 'args'


KeyboardInterrupt: 

In [44]:
event_signatures = {
    web3.keccak(text="NewDaoCreated(address,address,address[],uint256[],string,string,string,uint256,address,string[],string[])").hex(): "NewDaoCreated",
    "0x3134e8a2e6d97e929a7e54011ea5485d7d196dd5f0ba4d4ef95803e8e3fc257f":"DelegateChanged",
    "0x7d84a6263ae0d98d3329bd7b46bb4e8d6f98cd35a7adb45c274c8b7fd5ebd5e0":"ProposalCreated"
}

In [3]:
logs

[]

In [2]:
latest_block = 14359876
from_block = latest_block - 5
ceva=""
logs = web3.eth.get_logs({
    "fromBlock": from_block,
    "toBlock": latest_block,
    "address": contract_address,
    # "topics": [[*event_signatures.keys()],None]

})
if len(logs)>0:
    event_signature = log["topics"][0].hex()
    ceva=event_signature
for log in logs:
    # Decode the log
    event = web3.eth.contract(abi=daoAbiGlobal).events.ProposalCreated().process_log(log)
    decoded_log=event
    proposal_id = decoded_log["args"]["proposalId"]
    proposer = decoded_log["args"]["proposer"]
    address= decoded_log['address']
    targets = decoded_log["args"]["targets"]
    values = decoded_log["args"]["values"]
    signatures = decoded_log["args"]["signatures"]
    calldatas = decoded_log["args"]["calldatas"]
    vote_start = decoded_log["args"]["voteStart"]
    vote_end = decoded_log["args"]["voteEnd"]
    description = decoded_log["args"]["description"]
    parts = description.split("0|||0")
    # Extract the fields
    if len(parts) > 3:
        name = parts[0]
        type_ = parts[1]  # Using `type_` to avoid shadowing the built-in `type`
        desc = parts[2]
        link=parts[3]
    else:
        name = type_ = desc = link = None  # Handle case where the pattern is incomplete

logs


[AttributeDict({'address': '0x09166B4cA9572648B42A2A644D0c0daF4888Df48',
  'topics': [HexBytes('0x7d84a6263ae0d98d3329bd7b46bb4e8d6f98cd35a7adb45c274c8b7fd5ebd5e0')],
  'data': HexBytes('0x4cedecc4765c9d6fb35790494a61469f814bd9925aff9ac59fb8ea65c8bbb6bb000000000000000000000000a9f8f9c0bf3188ceddb9684ae28655187552bae90000000000000000000000000000000000000000000000000000000000000120000000000000000000000000000000000000000000000000000000000000016000000000000000000000000000000000000000000000000000000000000001a000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000db1d800000000000000000000000000000000000000000000000000000000000db1eac0000000000000000000000000000000000000000000000000000000000000340000000000000000000000000000000000000000000000000000000000000000100000000000000000000000060a93c29e3966c58a5227e1d76dcb185baa1ac6b00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000

In [4]:
event['args']

AttributeDict({'proposalId': 34796153355433061949199943255421441891766800982301842079298464375245943584443,
 'proposer': '0xa9F8F9C0bf3188cEDdb9684ae28655187552bAE9',
 'targets': ['0x60A93C29e3966c58a5227e1D76dcB185BAa1ac6b'],
 'values': [0],
 'signatures': [''],
 'calldatas': [b'%Y\xdd\xf5\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x07cevamic\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x17sivaloareamea vine aici\x00\x00\x00\x00\x00\x00\x00\x00\x00'],
 'voteStart': 14359936,
 'vote

In [12]:
from apps.homebase.entities import ProposalStatus, Proposal
p: Proposal = Proposal(name=name, org=address)
p.author = proposer
p.id = proposal_id
p.type = type_
p.targets = targets
p.values = values
p.callDatas = calldatas
from datetime import timezone
block_details = web3.eth.get_block(log.blockNumber)
p.createdAt = datetime.fromtimestamp(
    block_details['timestamp'], tz=timezone.utc)
p.votingStartsBlock = str(vote_start)
p.votingEndsBlock = str(vote_end)
p.externalResource = link

In [13]:
proposal_doc_ref = daos_collection \
    .document("0x09166B4cA9572648B42A2A644D0c0daF4888Df48") \
    .collection('proposals') \
    .document("34796153355433061949199943255421441891766800982301842079298464375245943584443")
proposal_doc_ref.set(p.toJson())

update_time {
  seconds: 1732833964
  nanos: 510510000
}

In [56]:
event_signatures

{'01c5013cf023a364cc49643b8f57347e398d2f0db0968edeb64e7c41bf2dfbde': 'NewDaoCreated',
 '0x3134e8a2e6d97e929a7e54011ea5485d7d196dd5f0ba4d4ef95803e8e3fc257f': 'DelegateChanged',
 '0x7d84a6263ae0d98d3329bd7b46bb4e8d6f98cd35a7adb45c274c8b7fd5ebd5e0': 'ProposalCreated'}

In [ ]:
event_signatures[f"0x{ceva}"]

'7d84a6263ae0d98d3329bd7b46bb4e8d6f98cd35a7adb45c274c8b7fd5ebd5e0'

In [ ]:
from apps.homebase.entities import Proposal
decoded_log=event
proposal_id = decoded_log["args"]["proposalId"]
proposer = decoded_log["args"]["proposer"]
address= decoded_log['address']
targets = decoded_log["args"]["targets"]
values = decoded_log["args"]["values"]
signatures = decoded_log["args"]["signatures"]
calldatas = decoded_log["args"]["calldatas"]
vote_start = decoded_log["args"]["voteStart"]
vote_end = decoded_log["args"]["voteEnd"]
description = decoded_log["args"]["description"]
parts = description.split("0|||0")

# Extract the fields
if len(parts) >= 3:
    name = parts[0]
    type_ = parts[1]  # Using `type_` to avoid shadowing the built-in `type`
    desc = parts[2]
    link=parts[3]
else:
    name = type_ = desc = link = None  # Handle case where the pattern is incomplete

p:Proposal= Proposal(name=name, org=address)
p.author=proposer
p.id=proposal_id
p.type=type_
p.targets=targets
p.values=values
p.callDatas=calldatas
from datetime import timezone
block_details = web3.eth.get_block(logs[0].blockNumber)
p.createdAt=datetime.fromtimestamp(block_details['timestamp'], tz=timezone.utc)
p.votingStartsBlock=str(vote_start)
p.votingEndsBlock=str(vote_end)
p.externalResource=link

Proposal ID: 34796153355433061949199943255421441891766800982301842079298464375245943584443
Proposer: 0xa9F8F9C0bf3188cEDdb9684ae28655187552bAE9
Targets: ['0x60A93C29e3966c58a5227e1D76dcB185BAa1ac6b']
Values: [0]
Signatures: ['']
Calldatas: ['2559ddf5000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000800000000000000000000000000000000000000000000000000000000000000007636576616d6963000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000017736976616c6f617265616d65612076696e652061696369000000000000000000']
Vote Start: 14359936
Vote End: 14360236
Description: Title0|||0registry0|||0description is longer0|||0


In [4]:
from datetime import timezone
block_details = web3.eth.get_block(14389936)
p.createdAt=datetime.fromtimestamp(block_details['timestamp'], tz=timezone.utc)

In [26]:
logs[0].blockNumber

14359876

In [24]:
vote_start

14359936

In [22]:
datetime.utcfromtimestamp(block_details['timestamp'])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_33284\1008997432.py:1: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  datetime.utcfromtimestamp(block_details['timestamp'])


datetime.datetime(2024, 11, 28, 13, 12, 23)

In [4]:
tabi=re.sub(r'\n+', ' ', tokenAbiGlobal).strip()
token_contract = web3.eth.contract(address="0xFdEe849bA09bFE39aF1973F68bA8A1E1dE79DBF9", abi=tabi)


# Call the read-only function


print(f"Address value: {address_variable_value}")


Address value: 2


In [8]:
type(address_variable_value)

int

In [23]:
first=web3.eth.block_number-4
logs = web3.eth.get_logs({
    "fromBlock": first,
    "toBlock": web3.eth.block_number,
    "address": wrapper_address,
    "topics": [
        web3.keccak(text="NewDaoCreated(address,address,address[],uint256[],string,string,string,uint256)").hex(),
        None,  # Match any dao address
    ]
})
len(logs)

0

In [24]:
first

13872535

In [9]:
decoded_event = wrapper_contract.events.NewDaoCreated().process_log(logs[0])

In [10]:
decoded_event

AttributeDict({'args': AttributeDict({'dao': '0x833365797370B3fE7F5B86E10FA62195e74b102e',
  'token': '0x69d7aF22AdDECA38b47686237c5a3d6Ce915111E',
  'initialMembers': ['0x36fE1912506E4b3B85e5a4e102237Ea81da1C3d3'],
  'initialAmounts': [100000, 1, 1, 1, 1],
  'name': 'aced',
  'symbol': 'ADS',
  'description': 'this is the description',
  'executionDelay': 90}),
 'event': 'NewDaoCreated',
 'logIndex': 11,
 'transactionIndex': 1,
 'transactionHash': HexBytes('0x36e1197c733317bed02a9cf6e9bb084ba8ccc5a04cdff9072c51a7fd9c46530c'),
 'address': '0xA3C99359F376bE32ADE1C259Faa01e119B038436',
 'blockHash': HexBytes('0x9570374a8a889a9c65db2b6c54d313caebfbbbac6ec02c14ad45089199c5f5f8'),
 'blockNumber': 13871169})

In [20]:
project_addresses = [doc.id for doc in projects_collection.stream()]
project_addresses

['0x379080EE8941CE5Ae5560d5E32A4CEBBaC7C6b92']

In [10]:
a=[1,2,3,4,5,6,7,8,9]
a[-2]

8

In [26]:

def listen_to_project_events(contract, contract_address):
    global processed_transactions
    
    # Define the topics for all events
    event_signatures = {
        "SetParties": web3.keccak(text="SetParties(address,address,string)").hex(),
        "SendFunds": web3.keccak(text="SendFunds(address,uint256)").hex(),
        "ContractorPaid": web3.keccak(text="ContractorPaid(address,uint256)").hex(),
        "ContributorWithdrawn": web3.keccak(text="ContributorWithdrawn(address,uint256)").hex(),
        "ProjectDisputed": web3.keccak(text="ProjectDisputed(address)").hex(),
        "ProjectClosed": web3.keccak(text="ProjectClosed(address)").hex(),
        "ContractSigned": web3.keccak(text="ContractSigned(address)").hex(),
        "ArbitrationDecision": web3.keccak(text="ArbitrationDecision(address,uint256,string)").hex(),
    }

    # Get logs for the current contract
    logs = web3.eth.get_logs({
        "fromBlock": web3.eth.block_number-3,
        "toBlock": web3.eth.block_number,
        "address": contract_address,
    })

    for log in logs:
        tx_hash = log["transactionHash"].hex()
        if tx_hash in processed_transactions:
            print("already did this one")
            continue  # Skip already processed transactions

        try:
            event_type = None
            for event_name, topic_hash in event_signatures.items():
                print("got one!")
                if log["topics"][0].hex() == topic_hash:
                    event_type = event_name
                    break

            if not event_type:
                print("Not an avent type")
                continue  # Skip logs that don't match any defined events

            # Decode and process the event
            decoded_event = getattr(contract.events, event_type)().process_log(log)
            event_data = getattr(ProjectEvents, event_type)(decoded_event)

            print(f"Event detected in contract {contract_address} ({event_type}):", event_data)

        except Exception as e:
            print(f"Error processing log for contract {contract_address}: {e}")

        # Mark the transaction as processed
        processed_transactions.add(tx_hash)


In [ ]:
while True:
    heartbeat += 1

    for address in project_addresses:
        # Get contract instance
        contract = web3.eth.contract(address=address, abi=native_project_abi)
        try:
            listen_to_project_events(contract, address)
        except Exception as e:
            
            print("Errrorrr "+e)

    if heartbeat % 10 == 0:
        print(f"Heartbeat: {heartbeat}")

    time.sleep(3)

In [ ]:
import apps.services

In [ ]:
import requests
dorg_homebase_channel_id = "804689936388325376"

def send_discord_message(msg, channel_id):
    global discord_bot_token
    url = "https://discordapp.com/api/channels/"+channel_id+"/messages"
    headers = {
        "Authorization": "Bot " + discord_bot_token
    }
    body = {
        "content": msg
    }
    print("before sending "+msg)
    response = requests.request("POST", url, headers=headers, data=body)
    return {"data": str("Status code "+str(response.status_code))}

In [31]:
send_discord_message(channel_id=dorg_homebase_channel_id, msg="INDEXER IS DOWN ON TRUSTLESS!")

before sending INDEXER IS DOWN ON TRUSTLESS!


{'data': 'Status code 200'}

In [33]:
from dotenv import dotenv_values
env_variables = dotenv_values(".env")

In [ ]:
my_variable = env_variables["discord_bot_token"]
print(my_variable)